In [8]:
import os
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
from finn.util.visualization import showInNetron
import numpy as np


In [2]:
model_dir = os.environ['FINN_ROOT'] + '/notebooks/projects/'
ready_model_filename = model_dir + '/model.onnx'
model_for_sim = ModelWrapper(ready_model_filename)

# 1. Expose model into FINN

In [3]:
print(dir(model_for_sim))

finnonnx_in_tensor_name  = model_for_sim.graph.input[0].name
finnonnx_out_tensor_name = model_for_sim.graph.output[0].name

print(f'Input Tensor name: {finnonnx_in_tensor_name}')
print(f'Output Tensor name: {finnonnx_out_tensor_name}')

finnonnx_model_in_shape  = model_for_sim.get_tensor_shape(finnonnx_in_tensor_name)
finnonnx_model_out_shape = model_for_sim.get_tensor_shape(finnonnx_out_tensor_name)

print(f'Input Tensor shape: {finnonnx_model_in_shape}')
print(f'Output Tensor shape: {finnonnx_model_out_shape}')

finnonnx_model_in_dt  = model_for_sim.get_tensor_datatype(finnonnx_in_tensor_name)
finnonnx_model_out_dt = model_for_sim.get_tensor_datatype(finnonnx_out_tensor_name)

print(f'Input Tensor datatype: {str(finnonnx_model_in_dt.name)}')
print(f'Output Tensor datatype: {str(finnonnx_model_out_dt.name)}')

print('List of node operator types in the graph: ')
print([x.op_type for x in model_for_sim.graph.node])


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_model_proto', 'analysis', 'check_all_tensor_shapes_specified', 'check_compatibility', 'cleanup', 'find_consumer', 'find_consumers', 'find_direct_predecessors', 'find_direct_successors', 'find_producer', 'find_upstream', 'fix_float64', 'get_all_tensor_names', 'get_finn_nodes', 'get_initializer', 'get_metadata_prop', 'get_node_from_name', 'get_node_index', 'get_nodes_by_op_type', 'get_non_finn_nodes', 'get_tensor_datatype', 'get_tensor_fanout', 'get_tensor_layout', 'get_tensor_shape', 'get_tensor_sparsity', 'get_tensor_valueinfo', 'graph', 'is_fork_node', 'is_join_node', 'make_empty_exec_context', 'make_new_valueinfo_name', 'model', 'rename_tensor

# 2. Tidy up Network transformations

In [6]:
model_for_sim = model_for_sim.transform(InferShapes())
model_for_sim = model_for_sim.transform(FoldConstants())
model_for_sim = model_for_sim.transform(GiveUniqueNodeNames())
model_for_sim = model_for_sim.transform(GiveReadableTensorNames())
model_for_sim = model_for_sim.transform(InferDataTypes())
model_for_sim = model_for_sim.transform(RemoveStaticGraphInputs())

verify_model_name = model_dir + '/verify.onnx'
model_for_sim.save(verify_model_name)

In [7]:
showInNetron(verify_model_name)

Serving '/home/petertso/finn/notebooks/projects//verify.onnx' at http://0.0.0.0:8081
